In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.0 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import math
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim
import warnings

from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from pathlib import Path
from typing import Any

from tqdm import tqdm

Здесь config нашей модели

In [ ]:
def get_config():
    return {
        "batch_size": 32,
        "num_epochs": 10,
        "lr": 2e-4,
        "seq_len": 570,
        "d_model": 512,
        "datasource": 'Helsinki-NLP/news_commentary',
        "lang_src": "en",
        "lang_tgt": "ru",
        "model_folder": "weights",
        "model_basename": "transformer_translate_model",
        "preload": None,
        "tokenizer_file": "tokenizer_{0}.json",
        "experimental_name": "runs/transformer_translate_model"
    }

def get_weights_file_path(config, epoch: str):
    model_folder = config['model_folder']
    model_basename = config['model_basename']
    model_filename = f"{model_basename}{epoch}.pt"
    return str(Path('.') / model_folder / model_filename)


Здесь сама модель трансформера

In [ ]:
# Input Embeddings
# Этот класс преобразует входные токены в векторные представления.
class InputEmbeddings(nn.Module):
    def __init__(self, d_model: int, vocab_size: int ):
        super().__init__()
        self.d_model = d_model                # Размерность эмбеддингов (например, 512)
        self.vocab_size = vocab_size          # Размер словаря
        # Создаем слой эмбеддингов: каждому токену соответствует вектор размера d_model
        self.embedding = nn.Embedding(vocab_size, d_model)  # d_model = 512 (размер эмбеддинга)

    def forward(self, x):
        # Умножаем эмбеддинги на корень из d_model для стабилизации градиентов
        return self.embedding(x) * math.sqrt(self.d_model)

# Positional Encodings
# Этот класс добавляет информацию о позиции каждого токена в последовательности.
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)

        # Создаем матрицу позиций размером (seq_len, d_model)
        # Используем логарифмическую шкалу для повышения стабильности вычислений
        pe = torch.zeros(seq_len, d_model)
        # Создаем вектор позиций (seq_len, 1)
        position = torch.arange(0, seq_len, dtype=torch.float32).unsqueeze(1)  # (seq_len, 1)
        # Вычисляем коэффициенты масштабирования для синусов и косинусов
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        # Применяем sin к четным индексам и cos к нечетным индексам
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # Добавляем дополнительное измерение, чтобы получить форму (1, seq_len, d_model)
        pe = pe.unsqueeze(0)
        # Регистрируем pe как буфер, чтобы он не обновлялся во время обучения
        self.register_buffer('pe', pe)

    def forward(self, x):
        # Прибавляем позиционные кодировки к входным эмбеддингам
        # .require_grad_(False) гарантирует, что этот тензор не будет обучаться
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False)
        return self.dropout(x)

# Layer Normalization
# Этот класс нормализует входной тензор по последнему измерению, чтобы ускорить обучение.
class LayerNormalization(nn.Module):
    def __init__(self, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        # Параметры, которые обучаются: масштаб (alpha) и смещение (bias)
        self.alpha = nn.Parameter(torch.ones(1))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        # Вычисляем среднее и стандартное отклонение по последнему измерению
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        # Нормализуем x и применяем обучаемые параметры
        return self.alpha * (x - mean) / (std + self.eps) + self.bias

# Feed Forward Block
# Этот блок представляет собой двухслойную полносвязную сеть с активацией и dropout.
class FeedForward(nn.Module):
    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        # Первый линейный слой преобразует размерность с d_model до d_ff
        self.linear1 = nn.Linear(d_model, d_ff)  # W1 и b1
        self.dropout = nn.Dropout(dropout)
        # Второй линейный слой возвращает размерность обратно к d_model
        self.linear2 = nn.Linear(d_ff, d_model)    # W2 и b2

    def forward(self, x):
        # Пропускаем x через первый слой, применяем ReLU, затем dropout и второй слой
        x = self.dropout(torch.relu(self.linear1(x)))
        x = self.linear2(x)
        return x

# Multi-Head Attention
# Этот блок реализует механизм многоголового внимания.
class MultiHeadAttentionBlock(nn.Module):
    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model                # Общая размерность модели
        self.h = h                            # Количество голов
        # Проверка: d_model должен быть кратен количеству голов
        assert d_model % h == 0, "d_model must be divisible by h"

        self.d_k = d_model // h               # Размерность каждой головы
        # Линейные слои для вычисления Q, K, V
        self.w_q = nn.Linear(d_model, d_model)  # Wq
        self.w_k = nn.Linear(d_model, d_model)  # Wk
        self.w_v = nn.Linear(d_model, d_model)  # Wv

        # Линейный слой для объединения голов после внимания
        self.w_o = nn.Linear(d_model, d_model)  # Wo
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        # Получаем размерность ключей (d_k)
        d_k = query.shape[-1]
        # Вычисляем оценки внимания с масштабированием
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        # Если маска задана, заполняем те позиции, где маска == 0, очень маленьким числом
        if mask is not None:
            attention_scores = attention_scores.masked_fill(mask == 0, -1e9)
        # Применяем softmax для получения вероятностей внимания
        attention_scores = attention_scores.softmax(dim=-1)
        # Если задан dropout, применяем его к оценкам внимания
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        # Возвращаем результат (взвешенное значение) и сами оценки внимания
        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        # Применяем линейные преобразования для Q, K, V
        query = self.w_q(q)  # (batch_size, seq_len, d_model)
        key = self.w_k(k)    # (batch_size, seq_len, d_model)
        value = self.w_v(v)  # (batch_size, seq_len, d_model)

        # Изменяем форму тензоров для разделения на головы
        # Преобразуем форму: (batch_size, seq_len, d_model) -> (batch_size, seq_len, h, d_k)
        # Затем транспонируем, чтобы получить: (batch_size, h, seq_len, d_k)
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        # Вычисляем механизм внимания и получаем выход и веса внимания
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)

        # Транспонируем обратно и объединяем головы в один тензор:
        # (batch_size, h, seq_len, d_k) -> (batch_size, seq_len, h, d_k) -> (batch_size, seq_len, d_model)
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

        # Пропускаем объединенный тензор через итоговый линейный слой
        return self.w_o(x)

# Residual Connection
# Этот класс реализует остаточные связи с нормализацией и dropout, чтобы стабилизировать обучение.
class ResidualConnection(nn.Module):
    def __init__(self, dropout: float):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.norm = LayerNormalization()

    def forward(self, x, sublayer):
        # Нормализуем вход, пропускаем через sublayer, применяем dropout и прибавляем исходный x (skip connection)
        return x + self.dropout(sublayer(self.norm(x)))

# Encoder Block
# Этот блок объединяет в себе слой самовнимания и feed-forward блок с остаточными связями.
class EncoderBlock(nn.Module):
    def __init__(self, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForward, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        # Создаем два остаточных подключения: одно для самовнимания, другое для feed-forward
        self.residual_connection = nn.ModuleList([ResidualConnection(dropout) for _ in range(2)])

    def forward(self, x, scr_mask):
        # Применяем остаточную связь для слоя самовнимания
        x = self.residual_connection[0](x, lambda x: self.self_attention_block(x, x, x, scr_mask))
        # Применяем остаточную связь для feed-forward блока
        x = self.residual_connection[1](x, self.feed_forward_block)
        return x

# Encoder
# Последовательность EncoderBlock'ов, завершающаяся финальной нормализацией.
class Encoder(nn.Module):
    def __init__(self, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()

    def forward(self, x, mask):
        # Проходим последовательно по всем блокам энкодера
        for layer in self.layers:
            x = layer(x, mask)
        # Применяем финальную нормализацию
        return self.norm(x)

# Decoder Block
# Этот блок декодера состоит из трёх частей: самовнимание, кросс-внимание и feed-forward, каждая с остаточной связью.
class DecoderBlock(nn.Module):
    def __init__(self, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForward, dropout:float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        # Создаем три остаточных подключения для каждого подблока декодера
        self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(3)])

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        # Остаточная связь для самовнимания с target mask
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        # Остаточная связь для кросс-внимания (между декодером и энкодером)
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        # Остаточная связь для feed-forward блока
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x

# Decoder
# Модуль декодера, состоящий из нескольких DecoderBlock'ов, завершающийся финальной нормализацией.
class Decoder(nn.Module):
    def __init__(self, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        # Последовательно проходим через все блоки декодера
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        # Финальная нормализация
        return self.norm(x)

# Projection Layer
# Преобразует выход декодера в вероятностное распределение по словарю.
class ProjectionLayer(nn.Module):
    def __init__(self, d_model: int, vocab_size: int ) -> None:
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        # Применяем линейное преобразование и логарифмический softmax для получения лог-вероятностей
        return torch.log_softmax(self.proj(x), dim=-1)

# Transformer
# Объединяет энкодер, декодер и эмбеддинги с позиционными кодировками в одну модель.
class Transformer(nn.Module):
    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings,
                 tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding,
                 projection_layer: ProjectionLayer):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer

    def encode(self, src, src_mask):
        # Применяем эмбеддинги и позиционные кодировки, затем энкодер
        src = self.src_embed(src)
        src = self.src_pos(src)
        return self.encoder(src, src_mask)

    def decode(self, encoder_output, src_mask, tgt, tgt_mask):
        # Применяем эмбеддинги и позиционные кодировки к таргету, затем декодер
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos(tgt)
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

    def project(self, x):
        # Преобразуем выход декодера в лог-вероятности по словарю
        return self.projection_layer(x)


# Функция для сборки модели Transformer с заданными гиперпараметрами.
def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int,
                      tgt_seq_len: int, d_model: int = 512, N: int = 6, h: int = 8,
                      dropout: float = 0.1, d_ff: int = 2048) -> Transformer:
    # Создаем эмбеддинги для исходного и целевого языков
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    # Создаем позиционные кодировки для исходного и целевого языков
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)

    # Собираем блоки энкодера
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForward(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    # Собираем блоки декодера
    decoder_blocks = []
    for _ in range(N):
        # Обратите внимание: в оригинальной архитектуре для декодера часто используются разные блоки для
        # самовнимания и кросс-внимания. Здесь для простоты используются одинаковые блоки.
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForward(d_model, d_ff, dropout)
        decoder_block = DecoderBlock(decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)

    # Инициализируем энкодер и декодер, используя ModuleList для последовательности блоков
    encoder = Encoder(nn.ModuleList(encoder_blocks))
    decoder = Decoder(nn.ModuleList(decoder_blocks))

    # Создаем проекционный слой для получения лог-вероятностей по словарю целевого языка
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

    # Собираем финальный Transformer
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

    # Инициализация параметров модели с помощью Xavier uniform
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return transformer


Здесь наш датасет и его обработка

In [ ]:
class BiLanguageDataset(Dataset):
    def __init__(self, ds, tokenizer_src, tokenizer_tgt, src_lang, tgt_lang, seq_len) -> None:
        # Здесь мы инициализируем наш датасет для двуязычных данных,
        # чтобы модель могла шибко круто учиться переводить тексты.
        super().__init__()
        self.ds = ds  # Наш супер датасет с парами переводов.
        self.tokenizer_src = tokenizer_src  # Токенизатор для исходного языка (например, английский).
        self.tokenizer_tgt = tokenizer_tgt  # Токенизатор для целевого языка (например, русский).
        self.src_lang = src_lang  # Код исходного языка.
        self.tgt_lang = tgt_lang  # Код целевого языка.
        self.seq_len = seq_len  # Максимальная длина последовательности, чтобы всё было аккуратно.

        # Получаем специальные токены: старт ([SOS]), конец ([EOS]) и паддинг ([PAD]),
        # чтобы модель понимала, когда начинать и заканчивать работу с текстом.
        self.sos_token = torch.tensor([tokenizer_src.token_to_id('[SOS]')], dtype=torch.int64)
        self.eos_token = torch.tensor([tokenizer_src.token_to_id('[EOS]')], dtype=torch.int64)
        self.pad_token = torch.tensor([tokenizer_src.token_to_id('[PAD]')], dtype=torch.int64)

    def __len__(self):
        # Возвращаем количество элементов в датасете – быстро
        return len(self.ds)

    def __getitem__(self, index: Any) -> Any:
        # Получаем пару переводов по индексу
        src_target_pair = self.ds[index]
        src_text = src_target_pair['translation'][self.src_lang]  # Исходный текст.
        tgt_text = src_target_pair['translation'][self.tgt_lang]  # Перевод.

        # Токенизируем тексты в последовательности токенов.
        enc_input_tokens = self.tokenizer_src.encode(src_text).ids  # Токены для энкодера.
        dec_input_tokens = self.tokenizer_tgt.encode(tgt_text).ids  # Токены для декодера.

        # Вычисляем количество токенов, которые надо добавить, чтобы итоговая длина стала ровно seq_len.
        # Для энкодера учитываем, что добавляем [SOS] и [EOS] токены.
        enc_num_padding_tokens = self.seq_len - len(enc_input_tokens) - 2
        # Для декодера добавляем только [SOS] (а [EOS] добавляется уже к меткам).
        dec_num_padding_tokens = self.seq_len - len(dec_input_tokens) - 1

        # Если предложение слишком длинное, то не сможем уместить его в seq_len — об этом сигнализируем.
        if enc_num_padding_tokens < 0 or dec_num_padding_tokens < 0:
            raise ValueError('Sentence is too long')

        # Формируем вход для энкодера: [SOS] + токены + [EOS] + заполнение (padding)
        encoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(enc_input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * enc_num_padding_tokens, dtype=torch.int64)
            ]
        )
        # Формируем вход для декодера: [SOS] + токены + заполнение (padding)
        decoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64)
            ]
        )
        # Формируем метки (label) для обучения: токены + [EOS] + заполнение (padding)
        label = torch.cat(
            [
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64)
            ]
        )

        # Проверяем, что длина каждой последовательности ровно seq_len
        assert encoder_input.size(0) == self.seq_len
        assert decoder_input.size(0) == self.seq_len
        assert label.size(0) == self.seq_len

        # Возвращаем словарь с входами, масками и оригинальными текстами
        return {
            'encoder_input': encoder_input,  # Вход для энкодера.
            'decoder_input': decoder_input,  # Вход для декодера.
            # Маска для энкодера: отмечаем реальные токены (без паддинга).
            'encoder_mask': (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int(),
            # Маска для декодера: учитываем и паддинг, и каузальную маску, чтобы не подсматривать будущее.
            'decoder_mask': (decoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int() & causal_mask(decoder_input.size(0)),
            'label': label,            # Метки для обучения.
            'src_text': src_text,      # Исходный текст, чтобы мы могли вспоминать, откуда всё началось.
            'tgt_text': tgt_text       # Целевой текст — результат, к которому стремимся.
        }

# Функция для создания каузальной маски, которая запрещает модели "подглядывать" будущее при генерации.
def causal_mask(size):
    # Создаем верхнюю треугольную матрицу с единицами выше диагонали, начиная с diagonal=1.
    mask = torch.triu(torch.ones(1, size, size), diagonal=1).type(torch.int)
    # Возвращаем булеву маску: True там, где можно смотреть (и учиться на ошибках), и False там, где нельзя.
    return mask == 0


Здесь обучение модели


In [ ]:
def greedy_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
    # Берем индексы специальных токенов для начала и конца перевода – [SOS] и [EOS]
    sos_idx = tokenizer_tgt.token_to_id('[SOS]')
    eos_idx = tokenizer_tgt.token_to_id('[EOS]')

    # Пропускаем источник через энкодер, чтобы получить его представление
    encoder_output = model.encode(source, source_mask)
    # Инициализируем декодер с токеном [SOS] – стартуем наше приключение перевода
    decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(source).to(device)

    # Генерируем перевод токен за токеном, пока не достигнем максимальной длины или не встретим [EOS]
    while True:
        if decoder_input.size(1) == max_len:
            break
        # Создаем каузальную маску для декодера, чтобы он не заглядывал в будущее
        decoder_mask = causal_mask(decoder_input.size(1)).type_as(source).to(device)

        # Пропускаем данные через декодер и получаем вероятность следующего токена
        out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)
        prob = model.project(out[:, -1])
        # Выбираем токен с наивысшей вероятностью
        _, next_word = torch.max(prob, dim=1)
        # Добавляем выбранный токен к уже сгенерированной последовательности
        decoder_input = torch.cat(
            [
                decoder_input,
                torch.empty(1, 1).type_as(source).fill_(next_word.item()).to(device)
            ], dim=1
        )

        # Если встретился [EOS] – завершаем генерацию
        if next_word == eos_idx:
            break
    return decoder_input.squeeze(0)


def run_validation(model, validation_ds, tokenizer_src, tokenizer_tgt, max_len, device, print_msg, global_state, writer, num_examples=2):
    # Переводим модель в режим валидации, чтобы градиенты не мешали нашему спокойствию
    model.eval()
    count = 0

    # Списки для хранения исходных текстов, эталонных и предсказанных переводов – чтобы сравнить и полюбоваться результатами
    source_texts = []
    expected = []
    predicted = []

    console_width = 80
    with torch.no_grad():
        # Проходим по валидационному датасету
        for batch in validation_ds:
            count += 1
            encoder_input = batch['encoder_input'].to(device)
            encoder_mask = batch['encoder_mask'].to(device)

            assert encoder_input.size(0) == 1, "Batch size must be 1 for validation"

            # Генерируем перевод с помощью жадного декодера
            model_out = greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device)
            current_src_text = batch['src_text'][0]  # Исходный текст из батча
            target_text = batch['tgt_text'][0]         # Правильный перевод
            model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy())

            # Сохраняем данные для последующего сравнения
            source_texts.append(current_src_text)
            expected.append(target_text)
            predicted.append(model_out_text)

            # Выводим в консоль результат для наслаждения прогрессом
            print_msg('-' * console_width)
            print_msg(f"Source: {current_src_text}")
            print_msg(f"Target: {target_text}")
            print_msg(f"Predicted: {model_out_text}")

            if count >= num_examples:
                break


def get_all_sentences(ds, lang):
    # Генератор, который последовательно возвращает каждое предложение для указанного языка
    for item in ds:
        yield item['translation'][lang]


def build_tokenizer(config, ds, lang):
    # Формируем путь к файлу токенизатора для данного языка
    tokenizer_path = Path(config['tokenizer_file'].format(lang))
    if not Path.exists(tokenizer_path):
        # Если токенизатора еще нет – создаем его с нуля
        tokenizer = Tokenizer(WordLevel(unk_token='[UNK]'))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(special_tokens=['[UNK]', '[PAD]', '[SOS]', '[EOS]'], min_frequence=2)
        tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer=trainer)
        # Сохраняем токенизатор, чтобы потом не тратить время на его пересоздание
        tokenizer.save(str(tokenizer_path))
    else:
        # Если файл уже существует – просто загружаем токенизатор
        tokenizer = Tokenizer.from_file(str(tokenizer_path))
    return tokenizer


def get_ds(config):
    # Загружаем датасет новостных комментариев
    ds_row = load_dataset('Helsinki-NLP/news_commentary', f'{config["lang_src"]}-{config["lang_tgt"]}', split='train')
    # Строим токенизаторы для исходного и целевого языков
    tokenizer_src = build_tokenizer(config, ds_row, config['lang_src'])
    tokenizer_tgt = build_tokenizer(config, ds_row, config['lang_tgt'])

    # Разбиваем датасет на обучающую и валидационную выборки (90%/10%)
    train_ds_size = int(0.9 * len(ds_row))
    val_ds_size = len(ds_row) - train_ds_size
    train_ds_raw, val_ds_raw = random_split(ds_row, [train_ds_size, val_ds_size])

    # Создаем экземпляры нашего датасета для обоих наборов данных
    train_ds = BiLanguageDataset(train_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])
    val_ds = BiLanguageDataset(val_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])

    # Вычисляем максимальную длину предложений для каждого языка
    max_len_src = 0
    max_len_tgt = 0
    for item in ds_row:
        src_ids = tokenizer_src.encode(item['translation'][config['lang_src']]).ids
        tgt_ids = tokenizer_src.encode(item['translation'][config['lang_tgt']]).ids
        max_len_src = max(max_len_src, len(src_ids))
        max_len_tgt = max(max_len_tgt, len(tgt_ids))
    print(f'Max length of source sentence: {max_len_src}')
    print(f'Max length of target sentence: {max_len_tgt}')

    # Создаем DataLoader'ы для удобного перебора данных: обучающего и валидационного
    train_dataloader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True)
    val_dataloader = DataLoader(val_ds, batch_size=1, shuffle=True)

    return train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt


def get_model(config, vocab_src_len, vocab_tgt_len):
    # Создаем трансформер для перевода
    model = build_transformer(vocab_src_len, vocab_tgt_len, config['seq_len'], config['seq_len'], config['d_model'])
    return model


def train_model(config):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')

    # Создаем папку для сохранения весов модели
    Path(config['model_folder']).mkdir(parents=True, exist_ok=True)

    # Загружаем датасеты и токенизаторы
    train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
    # Строим модель-переводчик и перемещаем ее на нужное устройство
    model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

    # Инициализируем логгер для TensorBoard
    writer = SummaryWriter(config['experimental_name'])

    # Настраиваем оптимизатор для модели – Adam
    optimizer = optim.Adam(model.parameters(), lr=config['lr'], eps=1e-9)

    initial_epoch = 0
    global_step = 0
    # Если задан путь к предобученной модели, загружаем её, чтобы не начинать все с нуля
    if config['preload']:
        model_filename = get_weights_file_path(config, config['preload'])
        print(f'Preloaded model: {model_filename}')
        state = torch.load(model_filename)
        initial_epoch = state['epoch'] + 1
        optimizer.load_state_dict(state['optimizer_state_dict'])
        global_step = state['global_step']

    # Определяем функцию потерь с учетом игнорирования [PAD]-токена и с label smoothing – для стабильного обучения
    loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1).to(device)

    # Основной цикл обучения по эпохам
    for epoch in range(initial_epoch, config['num_epochs']):
        batch_iterator = tqdm(train_dataloader, desc=f'Processing epoch {epoch:02d}')

        for batch in batch_iterator:
            model.train()  # Переводим модель в режим обучения

            # Переносим все данные на выбранное устройство
            encoder_input = batch['encoder_input'].to(device)
            decoder_input = batch['decoder_input'].to(device)
            encoder_mask = batch['encoder_mask'].to(device)
            decoder_mask = batch['decoder_mask'].to(device)

            # Прямой проход через модель: энкодер, декодер и проекция на словарь целевого языка
            encoder_output = model.encode(encoder_input, encoder_mask)
            decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)
            proj_output = model.project(decoder_output)

            label = batch['label'].to(device)

            # Вычисляем значение потерь
            loss = loss_fn(proj_output.view(-1, tokenizer_tgt.get_vocab_size()), label.view(-1))
            batch_iterator.set_postfix({f"loss": f"{loss.item():6.3f}"})

            # Логируем потери в TensorBoard
            writer.add_scalar('train_loss', loss.item(), global_step)
            writer.flush()

            loss.backward()  # Обратное распространение ошибки
            optimizer.step()  # Обновляем веса модели
            optimizer.zero_grad()  # Обнуляем градиенты для следующей итерации

            global_step += 1

        # После каждой эпохи проводим валидацию, чтобы убедиться, что мы идем в правильном направлении
        run_validation(
            model,
            val_dataloader,
            tokenizer_src,
            tokenizer_tgt,
            config['seq_len'],
            device,
            lambda msg: batch_iterator.write(msg),  # Функция для вывода сообщений
            global_step,                         # Глобальный шаг, как отсчет нашего  пути
            writer                               # Логгер
        )

        # Сохраняем модель после каждой эпохи
        model_filename = get_weights_file_path(config, f'{epoch:02d}')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'global_step': global_step
        }, model_filename)


if __name__ == '__main__':
    # Загружаем конфигурацию и запускаем обучение модели – да начнется наше приключение перевода!
    config = get_config()
    train_model(config)


Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/26.9k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/45.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/190104 [00:00<?, ? examples/s]

...

--------------------------------------------------------------------------------
Source: The opposition must build itself as a credible and attractive alternative to the Assad regime, and the regime’s international and regional critics must assist in that process.
Target: Оппозиция должна проявить себя как надежная и привлекательная альтернатива режиму Асада, а международные и региональные критики режима должны помогать в этом процессе.
Predicted: Оппозиция должна построить себе как , так и привлекательную альтернативу режиму Асада , и международные и региональные критики должны помочь в этом процессе .
--------------------------------------------------------------------------------
Source: Lastly, don't take too dogmatic a view of corruption and or the looting of the public coffers; they are part of the system.
Target: И, наконец – не судите слишком строго коррупцию и грабеж общественной казны – это часть системы.
Predicted: И , наконец , не следует слишком взгляд на коррупцию и госуд

Processing epoch 08: 100%|██████████| 5347/5347 [1:05:45<00:00,  1.36it/s, loss=2.637]


--------------------------------------------------------------------------------
Source: After all, while his policy of confrontation has generated high levels of violence, tolerance of the drug cartels corrupted state institutions, sowing the ground for the problem in the first place.
Target: В конце концов, в то время как его политика конфронтации привела к высокому уровню насилия, толерантность к наркокартелям привела к коррупции государственных учреждений, изначально создавая основу для проблем.
Predicted: В конце концов , в то время как его политика конфронтации привела к высокому уровню насилия , терпимость к государственных институтов , почву для проблемы в первом месте .
--------------------------------------------------------------------------------
Source: If you spin the bottle and create an eddy inside it, the water will flow out much faster and more smoothly.
Target: Если Вы будете вращать бутылку и создадите внутри нее воронку, вода вытечет намного быстрее и ровным потоко

Processing epoch 09: 100%|██████████| 5347/5347 [1:05:41<00:00,  1.36it/s, loss=2.522]


--------------------------------------------------------------------------------
Source: On April 29, 1865, this radical proposal scraped through the Massachusetts General Court (the state’s bi-cameral legislature), owing to intense lobbying and the goodwill generated by Harvard alumni’s distinguished service for the Union during the Civil War.
Target: 29 апреля 1865 года это радикальное предложение прошло через Главный суд штата Массачусетс (двухпалатный законодательный орган штата), вследствие интенсивного лоббирования и доброй воли, которые обеспечили заслуги выпускников Гарварда перед Союзом во времена Гражданской войны.
Predicted: 29 апреля , , данное радикальное предложение судом ( - законодательство государства ), вызванное интенсивным и , в для Союза во время гражданской войны .
--------------------------------------------------------------------------------
Source: So long as a large number of American troops remains as an occupying force, they serve as a recruiting tool for i

Source: It is generally agreed that China’s impressive economic achievements during the last three decades are largely the result of the radical reform of its economic system.
Target: В основном, все согласны с тем, что впечатляющие экономические достижения Китая за последние три десятилетия в большой степени являются результатом радикальной реформы его экономической системы.
Predicted: Это , в основном , отражает экономические последствия Китая в последние годы , в основном , в основном , в экономической реформы .
Processing epoch 01: 100%|██████████| 5347/5347 [1:05:40<00:00,  1.36it/s, loss=3.907]
--------------------------------------------------------------------------------
Source: But today, Khrushchev is remembered mostly for his contribution to the demise of Stalinism – and, via Mikhail Gorbachev, whose hero he was, ultimately for helping to bring about communism’s demise.
Target: Но сегодня, Хрущева помнят главным образом благодаря его вкладу в разоблачение и уничтожение сталинизма – а также, благодаря Михаилу Горбачеву, чьим героем он был, и, в конечном счете, благодаря тому, что опосредованно Хрущев помог вызвать упадок коммунизма.
Predicted: Но сегодня Хрущев , в основном , его вклад в – и , через Михаила Саакашвили , чья дочь , в конечном итоге , помогает о коммунизма .
--------------------------------------------------------------------------------
Source: If it can, then perhaps watching other countries suffer will help convince the local political elite to consent to adjustment.
Target: Если она сможет, тогда наблюдение за тем, как страдают другие страны, возможно, поможет убедить местную политическую элиту согласиться на преобразования.
Predicted: Если это может , то , что может произойти другие страны , , помогут убедить местные политические элиты к соглашению .
Processing epoch 02: 100%|██████████| 5347/5347 [1:05:42<00:00,  1.36it/s, loss=3.694]
--------------------------------------------------------------------------------
Source: (There is some new evidence, uncovered by Jim Pfaus at Concordia University in Canada, that desensitization may be affecting women consumers of pornography as well.)
Target: (Есть новые свидетельства, открытые Джимом Пфаусом из университета Конкордия в Канаде, что десенсибилизация может также влиять на женщин, потребляющих порнографию).
Predicted: ( Есть некоторые новые доказательства , в университете в Канаде , что может быть женщин , и ).
--------------------------------------------------------------------------------
Source: Nordbanken had become fully state-owned and a new management was put in place to restore the bank to viability.
Target: «Nordbanken» полностью перешел в собственность государства, и чтобы восстановить жизнеспособность банка, было назначено новое руководствоии.
Predicted: « » стал полностью государством и новым управлением , в банк для жизнеспособности .
Processing epoch 03: 100%|██████████| 5347/5347 [1:05:42<00:00,  1.36it/s, loss=3.263]
--------------------------------------------------------------------------------
Source: This applies to the world economy today.
Target: Это можно применить к сегодняшней мировой экономике.
Predicted: Это применимо к мировой экономике .
--------------------------------------------------------------------------------
Source: In short, they fear that all of the sacrifices made for price stability will have been in vain.
Target: Короче говоря, они опасаются, что все жертвы, принесенные ими во имя стабильности цен, окажутся напрасны.
Predicted: Короче говоря , они опасаются , что все жертвы , которые за стабильность цен , были .
Processing epoch 04: 100%|██████████| 5347/5347 [1:05:43<00:00,  1.36it/s, loss=3.142]
--------------------------------------------------------------------------------
Source: Absent further trade agreements, there is a big risk that the pace of globalization will slow, with profound consequences for global poverty and welfare.
Target: При дальнейшем отсутствии торговых соглашений существует большой риск, что темп глобализации замедлится с глубокими последствиями для глобальной бедности и благосостояния.
Predicted: При отсутствии дальнейших торговых соглашений существует большая вероятность того , что скорость глобализации замедлится , с серьезными последствиями для глобальной бедности и благосостояния .
--------------------------------------------------------------------------------
Source: China’s macroeconomic policies are probably still too “administrative.”
Target: Возможно, макроэкономическая политика Китая по-прежнему является слишком «административной».
Predicted: политика Китая , вероятно , тоже является « административной ».
Processing epoch 05: 100%|██████████| 5347/5347 [1:05:42<00:00,  1.36it/s, loss=2.854]
--------------------------------------------------------------------------------
Source: But there is also a growing realization that malaria is a disaster that must be addressed.
Target: Но наряду с этим, растет и осознание того, что с бедствием под названием малярия необходимо бороться.
Predicted: Но также растущее осознание того , что малярия – это катастрофа , которую необходимо решить .
--------------------------------------------------------------------------------
Source: Regardless of what the Saakashvili government claims about his opponents, our political orientation is not pro-Russian, but pro-Georgian.
Target: Вне зависимости от того, что правительство Саакашвили говорит о своих оппонентах, наша политическая ориентация не пророссийская, а прогрузинская.
Predicted: Независимо от того , что правительство Саакашвили заявляет о своих оппонентов , наша политическая ориентация не является , а .
Processing epoch 06: 100%|██████████| 5347/5347 [1:05:43<00:00,  1.36it/s, loss=2.837]
--------------------------------------------------------------------------------
Source: The IIF believes that GDP could be fully 5% lower after five years, with unemployment more than 7% higher.
Target: ИМФ считает, что ВВП после пяти лет может сократиться на целые 5%, а безработица может оказаться на 7% выше.
Predicted: полагает , что ВВП может быть в целых 5 % ниже , чем на пять лет , а безработица выше на 7 %.
--------------------------------------------------------------------------------
Source: Indeed, China is on the precipice of becoming an environmental wasteland.
Target: Действительно, Китай находится на гране превращения в экологическую свалку.
Predicted: Действительно , Китай находится на краю пропасти , став экологические .
Processing epoch 07: 100%|██████████| 5347/5347 [1:05:41<00:00,  1.36it/s, loss=2.593]
--------------------------------------------------------------------------------
Source: The opposition must build itself as a credible and attractive alternative to the Assad regime, and the regime’s international and regional critics must assist in that process.
Target: Оппозиция должна проявить себя как надежная и привлекательная альтернатива режиму Асада, а международные и региональные критики режима должны помогать в этом процессе.
Predicted: Оппозиция должна построить себе как , так и привлекательную альтернативу режиму Асада , и международные и региональные критики должны помочь в этом процессе .
--------------------------------------------------------------------------------
Source: Lastly, don't take too dogmatic a view of corruption and or the looting of the public coffers; they are part of the system.
Target: И, наконец – не судите слишком строго коррупцию и грабеж общественной казны – это часть системы.
Predicted: И , наконец , не следует слишком взгляд на коррупцию и государственной ; они являются частью системы .
Processing epoch 08: 100%|██████████| 5347/5347 [1:05:45<00:00,  1.36it/s, loss=2.637]
--------------------------------------------------------------------------------
Source: After all, while his policy of confrontation has generated high levels of violence, tolerance of the drug cartels corrupted state institutions, sowing the ground for the problem in the first place.
Target: В конце концов, в то время как его политика конфронтации привела к высокому уровню насилия, толерантность к наркокартелям привела к коррупции государственных учреждений, изначально создавая основу для проблем.
Predicted: В конце концов , в то время как его политика конфронтации привела к высокому уровню насилия , терпимость к государственных институтов , почву для проблемы в первом месте .
--------------------------------------------------------------------------------
Source: If you spin the bottle and create an eddy inside it, the water will flow out much faster and more smoothly.
Target: Если Вы будете вращать бутылку и создадите внутри нее воронку, вода вытечет намного быстрее и ровным потоком.
Predicted: Если вы и в ней , вода гораздо быстрее и гладко .
Processing epoch 09: 100%|██████████| 5347/5347 [1:05:41<00:00,  1.36it/s, loss=2.522]
--------------------------------------------------------------------------------
Source: On April 29, 1865, this radical proposal scraped through the Massachusetts General Court (the state’s bi-cameral legislature), owing to intense lobbying and the goodwill generated by Harvard alumni’s distinguished service for the Union during the Civil War.
Target: 29 апреля 1865 года это радикальное предложение прошло через Главный суд штата Массачусетс (двухпалатный законодательный орган штата), вследствие интенсивного лоббирования и доброй воли, которые обеспечили заслуги выпускников Гарварда перед Союзом во времена Гражданской войны.
Predicted: 29 апреля , , данное радикальное предложение судом ( - законодательство государства ), вызванное интенсивным и , в для Союза во время гражданской войны .
--------------------------------------------------------------------------------
Source: So long as a large number of American troops remains as an occupying force, they serve as a recruiting tool for insurgents.
Target: До тех пор пока большое количество американских войск находится в стране в качестве оккупационной силы, они являются идеальным орудием вербовки для мятежников.
Predicted: До тех пор , пока большое количество американских войск остается оккупационной силой , они служат инструментом для повстанцев .